In [1]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train_processed.csv"

# Load the latest version
train_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "buddycyph/train-processed",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", train_df.head())

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_308/1643445753.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  train_df = kagglehub.load_dataset(


First 5 records:    Score                                               Text  \
0      5  I received this product early from the seller!...   
1      5  *****<br />Numi's Collection Assortment Melang...   
2      5  I was very careful not to overcook this pasta,...   
3      5  Buying this multi-pack I was misled by the pic...   
4      5  These bars are so good! I loved them warmed up...   

                                      Text_Processed  
0  receive product early seller tastey great midd...  
1  br numis collection assortment melange include...  
2  careful overcook pasta make sure take bite eve...  
3  buying multipack mislead picture whole hazel n...  
4  bar good love warm definitely think great snac...  


In [ ]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Step 1: Prepare Data
texts_train = train_df['Text_Processed'].tolist()
labels_train = train_df['Score'].tolist()

# Create a Dataset object
train_dataset = Dataset.from_dict({"text": texts_train, "label": labels_train})

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenizing the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)

E0000 00:00:1743853774.944671     601 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import tensorflow as tf

# Initialize the TPU system
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Automatically detects the TPU
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

In [ ]:
# Define the distribution strategy
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
from transformers import TFDistilBertForSequenceClassification

# Build the model inside the TPU strategy scope
with tpu_strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=['accuracy'])

In [ ]:
# Convert the dataset to a TensorFlow-compatible format
def to_tf_dataset(tokenized_data):
    return tf.data.Dataset.from_tensor_slices((
        {
            "input_ids": tokenized_data["input_ids"],
            "attention_mask": tokenized_data["attention_mask"]
        },
        tokenized_data["label"]
    ))

# Create the TensorFlow dataset
train_tf_dataset = to_tf_dataset(tokenized_train)

# Optimize the dataset pipeline
train_tf_dataset = train_tf_dataset.batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Train the model on TPU
EPOCHS = 3

# Calculate steps per epoch based on the dataset size and batch size
steps_per_epoch = len(texts_train) // 16

model.fit(train_tf_dataset, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)